In [3]:
import sklearn
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [39]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def get_metrics(y_true, y_pred):
    """ Compute various regression metrics

    Args:
        y_true: a column of the true values.
        y_pred: a column of the predicted values.
    
    Returns:
        A dictionary of regression metrics.
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return {
        'mean_squared_error': mse,
        'root_mean_squared_error': rmse,
        'mean_absolute_error': mae,
        'r_squared': r2
    }


# Tree Grid Search

In [8]:
Salary = pl.read_csv('./Data/Salary_Bball.csv')

SalaryTrain = Salary.filter(pl.col('Training-Validation') == 'Training')

SalaryTest = Salary.filter(pl.col('Training-Validation') == 'Validation')

Salary

playerID,yearID,salary,stint,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old,W,L,G_right,GS,CG,SHO,SV,IPouts,H_right,ER,HR_right,BB_right,SO_right,BAOpp,ERA,IBB_right,WP,HBP_right,BK,BFP,GF,R_right,SH_right,SF_right,GIDP_right,GS_right,InnOuts,PO,A,E,DP,PB,WP_right,SB_right,CS_right,ZR,Training-Validation
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""pillbr01""",2012,480000,1,48,0,105,10,22,3,0,4,11,1,0,6,19,1,2,1,0,5,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""hamptmi01""",2009,2000000,1,21,0,37,6,12,1,0,1,7,0,0,2,8,0,0,1,1,0,0,7,10,21,21,0,0,0,336,128,66,13,46,74,0.298,5.3,6,1,2,0,494,0,71,11,5,13,21,336,9,17,2,1,0,0,0,0,0,"""Training"""
"""mathisc01""",2007,380000,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""hansoto01""",2013,3725000,1,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,15,13,0,0,0,219,83,44,10,30,56,0.286,5.42,3,6,3,0,327,1,47,0,4,3,13,219,7,7,1,0,0,0,0,0,0,"""Training"""
"""wrighja01""",2006,800000,1,35,0,47,4,12,2,0,0,1,0,0,0,11,0,0,5,0,1,0,6,10,34,21,0,0,0,468,167,90,16,64,79,0.282,5.19,4,6,10,0,676,2,95,5,4,24,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""espinda01""",2015,1800000,1,118,0,367,59,88,21,1,13,37,5,2,33,106,5,6,3,3,6,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""pradoma01""",2016,11000000,1,153,0,600,70,183,37,3,8,75,2,2,49,69,4,4,0,5,24,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Validation"""
"""derosma01""",2013,750000,1,88,0,204,23,48,12,1,7,36,0,0,28,49,1,1,0,3,8,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""


In [10]:
Salary_Stats = SalaryTrain.select(pl.col('G'), pl.col('ZR'))
Salary = SalaryTrain.select(pl.col('salary'))
Salary_Stats_Test = SalaryTest.select(pl.col('G'), pl.col('ZR'))
Salary_Test = SalaryTest.select(pl.col('salary'))

In [2]:
param_grid = {'criterion': ['squared_error', 'friedman_mse'],
              'max_depth': [10, 20, None],
              'min_samples_split': [2, 10],
              'min_samples_leaf': [1, 5],
            }

param_grid

{'criterion': ['squared_error', 'friedman_mse'],
 'max_depth': [10, 20, None],
 'min_samples_split': [2, 10],
 'min_samples_leaf': [1, 5]}

In [13]:
tree_grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, verbose=3, scoring='neg_mean_squared_error')

tree_grid_search.fit(Salary_Stats, Salary)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-17711958331408.414 total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-21181588393923.941 total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-19745676391065.465 total time=   0.0s
[CV 4/5] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-19572491886427.148 total time=   0.0s
[CV 5/5] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-20585525959597.426 total time=   0.0s
[CV 1/5] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=10;, score=-17711958331408.414 total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=10;, score=-2

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [10, 20, None],
                         'min_samples_leaf': [1, 5],
                         'min_samples_split': [2, 10]},
             scoring='neg_mean_squared_error', verbose=3)

In [14]:
tree_grid_search.best_score_

np.float64(-19758817123434.168)

In [16]:
tree_grid_search.best_params_

{'criterion': 'squared_error',
 'max_depth': 10,
 'min_samples_leaf': 5,
 'min_samples_split': 2}

In [18]:
tree_grid_search.best_estimator_

DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)

In [20]:
Salary_Pred = tree_grid_search.predict(Salary_Stats)

Salary_Pred

array([2210760.8989899 , 3500929.81481481, 2076944.47959184, ...,
       2800565.625     , 3357841.08      , 2516874.28571429])

In [42]:
get_metrics(Salary, Salary_Pred)

{'mean_squared_error': np.float64(18902150587093.652),
 'root_mean_squared_error': np.float64(4347660.357835425),
 'mean_absolute_error': np.float64(3084256.5982865333),
 'r_squared': 0.10404366876133031}

# Forest Grid Search

In [54]:
rf_param_grid = {'criterion': ['squared_error', 'friedman_mse'],
                 'n_estimators': [10, 100],
              'max_depth': [None, 1,2,3,4,5],
              'min_samples_split' : [1, 5, 10],
               'min_samples_leaf' : [1, 5, 10]
              }

rf_param_grid

{'criterion': ['squared_error', 'friedman_mse'],
 'n_estimators': [10, 100],
 'max_depth': [None, 1, 2, 3, 4, 5],
 'min_samples_split': [1, 5, 10],
 'min_samples_leaf': [1, 5, 10]}

In [55]:
rf_grid_search = GridSearchCV(RandomForestRegressor(), rf_param_grid, verbose=3, scoring='neg_mean_squared_error')

rf_grid_search.fit(Salary_Stats, Salary)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV 1/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 4/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 5/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 1/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=100;, score=nan total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_sampl

C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionW

[CV 5/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10;, score=-20726620250655.734 total time=   0.0s


C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV 1/5] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=-17739777164354.605 total time=   0.3s


...

[CV 4/5] END criterion=friedman_mse, max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=100;, score=-19177676380533.512 total time=   0.2s


C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV 5/5] END criterion=friedman_mse, max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=100;, score=-20422402531133.133 total time=   0.1s


C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [None, 1, 2, 3, 4, 5],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [1, 5, 10],
                         'n_estimators': [10, 100]},
             scoring='neg_mean_squared_error', verbose=3)

In [56]:
rf_grid_search.best_score_

np.float64(-19376982947606.637)

In [57]:
rf_grid_search.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 4,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 100}

In [58]:
rf_grid_search.best_estimator_

RandomForestRegressor(criterion='friedman_mse', max_depth=4, min_samples_leaf=5,
                      min_samples_split=10)

In [59]:
Salary_Forest_Pred = rf_grid_search.predict(Salary_Stats)

Salary_Forest_Pred

array([2115090.39597446, 2934477.376129  , 2120293.01140622, ...,
       2786473.40959405, 3763180.92391425, 2753812.83861403])

In [60]:
get_metrics(Salary, Salary_Forest_Pred)

{'mean_squared_error': np.float64(19114983442028.418),
 'root_mean_squared_error': np.float64(4372068.554131833),
 'mean_absolute_error': np.float64(3111696.2372879577),
 'r_squared': 0.09395545456603094}

In [61]:
rf_grid_search.best_score_ > tree_grid_search.best_score_

np.True_

In [62]:
Winning_Model = rf_grid_search.predict(Salary_Stats_Test)

In [63]:
get_metrics(Salary_Test, Winning_Model)

{'mean_squared_error': np.float64(32907968252781.043),
 'root_mean_squared_error': np.float64(5736546.718434448),
 'mean_absolute_error': np.float64(3806089.150284626),
 'r_squared': 0.05405245166357975}